# Configuration

In [1]:
# Parameters
ENABLE_COLAB = False

PROJECT_NAME = 'ML1010-Group-Project'
#EXPERIMENT_NAME = 'ReviewText_Lemma_Bert2 (Logistic Regression)'
#FILE_NAME = '01_ML1010_GP_LR_Bert2'
#LOAD_FROM_EXP = False

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni//ML_Root/project_root'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

# Bootstrap Environment

In [2]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)

import mv_python_utils as mvutils


Wha...where am I?
I am awake now.

I have set your current working directory to /home/magni/ML_Root/project_root/ML1010-Group-Project
The current time is 20:24
Hello sir. I hope you had dinner.



# Setup Runtime Environment

In [3]:
#%%capture
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q 
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt

!python -m spacy download en_core_web_sm

nltk.download('stopwords')
%matplotlib inline

import cw_df_metric_utils as cwutils

'Google Colab not enabled'

2022-01-24 20:24:26.569834: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-24 20:24:26.569858: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 13.9 MB 9.4 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to /home/magni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-01-24 20:24:32.085664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-24 20:24:32.085687: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load Data

In [4]:
### load the meta data
jarvis.showProjectDataFiles()
data = []
with gzip.open(jarvis.DATA_DIR + '/01_original/Cell_Phones_and_Accessories.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

Here are all your project data files
[D] /home/magni//ML_Root/project_root/data/ML1010-Group-Project [Empty directory]

[D] /home/magni//ML_Root/project_root/data/ML1010-Group-Project/01_original
---[   gz][ json]--> Cell_Phones_and_Accessories.json.gz (1.15 GB)
---[   gz][ json]--> Cell_Phones_and_Accessories_5.json.gz (161.24 MB)
---[   gz][ json]--> meta_Cell_Phones_and_Accessories.json.gz (343.33 MB)

[D] /home/magni//ML_Root/project_root/data/ML1010-Group-Project/02_working
[*][  pkl]---------> 01_Cellphone_small.pkl (45.46 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_1.pkl.gz (6.88 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_2.pkl.gz (170.55 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_3.pkl.gz (295.59 MB)
[*][  pkl]---------> 01_NLP_ReviewText_small.pkl (28.94 MB)
[*][  pkl]---------> 01_NLP_Summary_small.pkl (3.82 MB)
[*][  pkl]---------> 01_NLP_Title_small.pkl (2.73 MB)
---[   gz][  pkl]--> 01_NL_ReviewText_All(new).pkl.gz (821.15 MB)
---[   gz][  pkl]--> 01_NL

In [5]:
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)

print(len(df))

10063255


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10063255 entries, 0 to 10063254
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   reviewerName    object 
 6   reviewText      object 
 7   summary         object 
 8   unixReviewTime  int64  
 9   vote            object 
 10  image           object 
 11  style           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 854.1+ MB


In [7]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format="%m %d, %Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10063255 entries, 0 to 10063254
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   overall         float64       
 1   verified        bool          
 2   reviewTime      datetime64[ns]
 3   reviewerID      object        
 4   asin            object        
 5   reviewerName    object        
 6   reviewText      object        
 7   summary         object        
 8   unixReviewTime  int64         
 9   vote            object        
 10  image           object        
 11  style           object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 854.1+ MB


In [8]:
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5.0,False,2014-11-19,A1GG51FWU0XQYH,098949232X,Paul Williams,If your into space this is the Calendar for you.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,False,2014-11-19,AVFIDS9RK38E0,098949232X,Sean Powell,Awesome pictures!,Five Stars,1416355200,NaN,NaN,NaN
2,5.0,False,2014-11-19,A2S4AVR5SJ7KMI,098949232X,Tom Davis,Great wall art and information for space explo...,Five Stars,1416355200,NaN,NaN,NaN
3,5.0,False,2014-11-19,AEMMMVOR9BFLI,098949232X,Kwajmeck,"As always, it is a quality calendar full of ve...",I love it. I buy a new one every year,1416355200,NaN,NaN,NaN
4,5.0,False,2014-11-19,A2DZXMBTY7KLYP,098949232X,ScottG43,This is a fantastic calendar. This is my third...,Great Calendar.,1416355200,NaN,NaN,NaN


In [9]:
### load the meta data
dataMeta = []
with gzip.open(jarvis.DATA_DIR + '/01_original/meta_Cell_Phones_and_Accessories.json.gz') as f:
    for l in f:
        dataMeta.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(dataMeta))

# first row of the list
print(dataMeta[0])

590071
{'category': ['Cell Phones & Accessories', 'Accessories', 'Screen Protectors'], 'tech1': '', 'description': [], 'fit': '', 'title': 'Trollies Sing a Long Party VHS', 'also_buy': [], 'tech2': '', 'brand': '', 'feature': [], 'rank': '557,379 in Movies & TV (', 'also_view': [], 'details': {'ASIN: ': '6303195164'}, 'main_cat': 'Movies & TV', 'similar_item': '', 'date': '', 'price': '', 'asin': '6303195164', 'imageURL': [], 'imageURLHighRes': []}


In [10]:
# convert list into pandas dataframe
dfMeta = pd.DataFrame.from_dict(dataMeta)

print(len(dfMeta))

590071


In [11]:
dfMeta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590071 entries, 0 to 590070
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         590071 non-null  object
 1   tech1            590071 non-null  object
 2   description      590071 non-null  object
 3   fit              590071 non-null  object
 4   title            590071 non-null  object
 5   also_buy         590071 non-null  object
 6   tech2            590071 non-null  object
 7   brand            590071 non-null  object
 8   feature          590071 non-null  object
 9   rank             590071 non-null  object
 10  also_view        590071 non-null  object
 11  details          590071 non-null  object
 12  main_cat         590071 non-null  object
 13  similar_item     590071 non-null  object
 14  date             590071 non-null  object
 15  price            590071 non-null  object
 16  asin             590071 non-null  object
 17  imageURL  

In [12]:
dfMeta.head(5)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Cell Phones & Accessories, Accessories, Scree...",,[],,Trollies Sing a Long Party VHS,[],,,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,,,,6303195164,[],[]
1,"[Cell Phones & Accessories, Accessories, Car A...",,[],,La Pura VHS,[],,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,,,,630524412X,[],[]
2,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[New items to give you a new good mood, welcom...",,Leopard Thanksgiving Easter PC Black Case for ...,[],,,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,,,,6310200240,[],[]
3,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,Puppies Faceplate Hard Case Protector for Net1...,[],,Live2Pedal,[Protect your phone with style through this At...,"[>#6,368,825 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501710,[],[]
4,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,White Wolf Faceplate Protector Hard Case for S...,[],,wirelessoutletusa,[Protect your phone with style through this At...,"[>#2,672,454 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501273,[],[]


In [13]:
dfMerge = pd.merge(df, dfMeta, how="inner", on=["asin"])

In [ ]:
dfMerge.info(memory_usage='deep')

In [14]:
dfMerge['category'] = dfMerge['category'].astype("string")
#dfMerge['style'] = dfMerge['style'].astype("string")
dfMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10071535 entries, 0 to 10071534
Data columns (total 30 columns):
 #   Column           Dtype         
---  ------           -----         
 0   overall          float64       
 1   verified         bool          
 2   reviewTime       datetime64[ns]
 3   reviewerID       object        
 4   asin             object        
 5   reviewerName     object        
 6   reviewText       object        
 7   summary          object        
 8   unixReviewTime   int64         
 9   vote             object        
 10  image            object        
 11  style            object        
 12  category         string        
 13  tech1            object        
 14  description      object        
 15  fit              object        
 16  title            object        
 17  also_buy         object        
 18  tech2            object        
 19  brand            object        
 20  feature          object        
 21  rank             object      

In [ ]:
dfMerge.head(5)

# Merged Data Explore

In [15]:
dfTemp = df
display(str(dfTemp.shape))
dfDeDupe = dfTemp.drop_duplicates(subset=['reviewerID', 'asin', 'reviewText', 'reviewTime'],keep=False)
display(str(dfDeDupe.shape))

'(10063255, 12)'

'(10020286, 12)'

In [16]:
cat_One = dfMerge.groupby(['main_cat']).size().to_frame('reviewCnt')
cat_One = cat_One.sort_values(by='reviewCnt', ascending=False)        
display(cat_One.shape)    
# show top records
cat_One.head(29)

(36, 1)

,reviewCnt
main_cat,
Cell Phones & Accessories,9238625
All Electronics,384002
Computers,132097
Sports & Outdoors,40105
Amazon Home,37667
Camera & Photo,28782
Home Audio & Theater,24440
Books,20581
Portable Audio & Accessories,19883


In [17]:
# group by category
cat_Two = dfMerge.groupby(['category']).size().to_frame('reviewCnt')
cat_Two = cat_Two.sort_values(by='reviewCnt', ascending=False)        
display(cat_Two.shape)
# show top records
cat_Two.head(133)

(1660, 1)

,reviewCnt
category,
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Basic Cases']",2331144
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves']",1650735
"['Cell Phones & Accessories', 'Accessories', 'Screen Protectors']",947013
[],645418
"['Cell Phones & Accessories', 'Accessories', 'Cables']",456917
...,...
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Basic Cases', 'Imported', '100% brand new and high quality.']",40
"['Cell Phones & Accessories', 'Accessories', 'Chargers & Power Adapters', 'Wall Chargers', 'Plastic', 'Charges handsets, tablets, and other USB devices at max rate for increased charging efficiency', 'Capacity to charge any two devices simultaneously; no need to wait for a port to be free', 'Short circuit and over charge protection for safety', 'Folding AC prongs for portability', 'Cables sold separately']",39
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Basic Cases', 'Compatible for Samsung Galaxy NOTE 5', 'Material: PC,Rhinestone', 'It protects your Phone back and frame from Fingerprints, Scratches, Dusts, Collisions And Abrasion', 'Easy access to all buttons, controls & ports without having to remove the case', ""It Perfectly matches with your Phone's shape""]",39


In [18]:
# group by category
cat_Three = dfMerge.groupby(['title']).size().to_frame('reviewCnt')
cat_Three = cat_Three.sort_values(by='reviewCnt', ascending=False)        
display(cat_Three.shape)
# show top 5 records
cat_Three.head(5)

(572547, 1)

,reviewCnt
title,
Anker Astro E1 5200mAh Candy bar-Sized Ultra Compact Portable Charger (External Battery Power Bank) with High-Speed Charging PowerIQ Technology (Black),17732
"Portable Chargers 16750 RAVPower 16750mAh External Battery Pack 4.5A Dual USB Output External Phone Charger Battery Bank Power Bank (iSmart 2.0 Tech) for Nintendo Switch, iPhone 7, Galaxy S8 - White",13560
"Anker PowerCore+ Mini, 3350mAh Lipstick-Sized Portable Charger (Premium Aluminum Power Bank), One of The Most Compact External Batteries, Compatible with iPhone Xs/XR, Android Smartphones and More",13248
"Anker PowerCore 20100 - Ultra High Capacity Power Bank with 4.8A Output, PowerIQ Technology for iPhone, iPad and Samsung Galaxy and More (Black)",13063
"Maxboost iPhone 6 Screen Protector, [3 Pack ] iPhone 6 Tempered Glass Screen Protectors 0.2mm Ballistic Glass For Apple iPhone 6 4.7 inch ONLY and Work with most Protective Case",11227


In [19]:
# group by category, count distinct subreddits and posts
cat_All = dfMerge.groupby(['main_cat','category', 'title']).size().to_frame('reviewCnt')
cat_All = cat_All.sort_values(by='reviewCnt', ascending=False)
cat_All = cat_All.reset_index()
display(cat_All.shape)    
# show top 5 records
cat_All.head(5)

(574417, 4)

,main_cat,category,title,reviewCnt
0,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",Anker Astro E1 5200mAh Candy bar-Sized Ultra C...,17732
1,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",Portable Chargers 16750 RAVPower 16750mAh Exte...,13560
2,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...","Anker PowerCore+ Mini, 3350mAh Lipstick-Sized ...",13248
3,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",Anker PowerCore 20100 - Ultra High Capacity Po...,13063
4,All Electronics,"['Cell Phones & Accessories', 'Accessories', '...","Maxboost iPhone 6 Screen Protector, [3 Pack ] ...",11227


In [20]:
cat_All['category'].nunique()

1660

In [21]:
#Trim and fix column Dtypes for saving to database
value_list=["['Cell Phones & Accessories', 'Cell Phones']",
           "['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']",
           "['Cell Phones & Accessories', 'Cell Phones', 'Carrier Cell Phones']" 
           ]
#Trim records
dfTrim = dfMerge.loc[(dfMerge.category.isin(value_list))]
#dfTrim = dfTrim.drop(dfTrim.columns[5], axis=1) #drop the Style column
dfTrim.reset_index(inplace=True, drop=True)

#change the review to a date time
dfTrim['reviewTime'] = pd.to_datetime(dfTrim['reviewTime'], format="%m %d, %Y")

#Convert all columns that are objects to string.
#having an issue writing it to the database with type issues. 
#objectColumns = list(dfTrim.select_dtypes(['object']))
#dfTrim[objectColumns] = dfTrim[objectColumns].astype("string")

dfTrim.info()                    

/home/magni/python_env/ML1010_env2/lib64/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512237 entries, 0 to 512236
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   overall          512237 non-null  float64       
 1   verified         512237 non-null  bool          
 2   reviewTime       512237 non-null  datetime64[ns]
 3   reviewerID       512237 non-null  object        
 4   asin             512237 non-null  object        
 5   reviewerName     512204 non-null  object        
 6   reviewText       511900 non-null  object        
 7   summary          512067 non-null  object        
 8   unixReviewTime   512237 non-null  int64         
 9   vote             105628 non-null  object        
 10  image            4444 non-null    object        
 11  style            217187 non-null  object        
 12  category         512237 non-null  string        
 13  tech1            512237 non-null  object        
 14  description      512

In [ ]:
dfTrim.head(5)

# Export option exploration

In [ ]:
#import sqlite3

#con = sqlite3.connect('trimmed_cellphone.db')
#dfTrim.to_sql('src_data', con=con, if_exists='replace')
#con.commit()
#con.close()
#DB Size is 2.14GB

In [ ]:
#dfTrim.to_csv('trimmed_cellphone.csv.gz', compression='gzip')
#GZip csv is 224MB

In [ ]:
#dfTrim.to_pickle('trimmed_cellphone.pkl')
#296.4MB

# Data Cleaning

In [22]:
pd.set_option('mode.chained_assignment', None)
dfTrim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512237 entries, 0 to 512236
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   overall          512237 non-null  float64       
 1   verified         512237 non-null  bool          
 2   reviewTime       512237 non-null  datetime64[ns]
 3   reviewerID       512237 non-null  object        
 4   asin             512237 non-null  object        
 5   reviewerName     512204 non-null  object        
 6   reviewText       511900 non-null  object        
 7   summary          512067 non-null  object        
 8   unixReviewTime   512237 non-null  int64         
 9   vote             105628 non-null  object        
 10  image            4444 non-null    object        
 11  style            217187 non-null  object        
 12  category         512237 non-null  string        
 13  tech1            512237 non-null  object        
 14  description      512

In [23]:
#Step 1 - Add in column for GUID
import uuid
dfTrim['uuid'] = [uuid.uuid4() for _ in range(len(dfTrim.index))]

dfTrim.uuid.duplicated().sum()

0

In [24]:
#Step 2 - Add in year column
dfTrim['reviewYear'] = dfTrim['reviewTime'].dt.year

In [25]:
#Step 3 - Trim to only chosen columns
columns = ['uuid', 'overall', 'verified', 'reviewTime', 'reviewYear',
            'reviewerID', 'asin', 'style', 'reviewText',
            'summary', 'vote','category',  'description',
            'title', 'brand', 'feature',  'main_cat','price']
dfExport = dfTrim[columns]
dfExport.info()
display(dfTrim.shape)
dfExport.drop_duplicates
display(dfTrim.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512237 entries, 0 to 512236
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   uuid         512237 non-null  object        
 1   overall      512237 non-null  float64       
 2   verified     512237 non-null  bool          
 3   reviewTime   512237 non-null  datetime64[ns]
 4   reviewYear   512237 non-null  int64         
 5   reviewerID   512237 non-null  object        
 6   asin         512237 non-null  object        
 7   style        217187 non-null  object        
 8   reviewText   511900 non-null  object        
 9   summary      512067 non-null  object        
 10  vote         105628 non-null  object        
 11  category     512237 non-null  string        
 12  description  512237 non-null  object        
 13  title        512237 non-null  object        
 14  brand        512237 non-null  object        
 15  feature      512237 non-null  obje

(512237, 32)

(512237, 32)

In [26]:
#Step 4 - Remove identified data we don't want
#identified in python book 02 during data column explore
value_list=["Cell Phones & Accessories"] 
#Trim records
dfExport = dfExport.loc[(dfExport.main_cat.isin(value_list))]
dfExport.reset_index(inplace=True, drop=True)
dfExport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506922 entries, 0 to 506921
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   uuid         506922 non-null  object        
 1   overall      506922 non-null  float64       
 2   verified     506922 non-null  bool          
 3   reviewTime   506922 non-null  datetime64[ns]
 4   reviewYear   506922 non-null  int64         
 5   reviewerID   506922 non-null  object        
 6   asin         506922 non-null  object        
 7   style        215656 non-null  object        
 8   reviewText   506592 non-null  object        
 9   summary      506757 non-null  object        
 10  vote         104004 non-null  object        
 11  category     506922 non-null  string        
 12  description  506922 non-null  object        
 13  title        506922 non-null  object        
 14  brand        506922 non-null  object        
 15  feature      506922 non-null  obje

In [27]:
#Step 5 - Drop null values

#drop all nulls in reviewText
dfExport = dfExport.dropna(subset=['reviewText']) 

#drop all nulls in "overall". Shouldn't be null but in case data changes
dfExport = dfExport.dropna(subset=['overall']) 

#drop all nulls in "summary". Shouldn't be null but in case data changes
dfExport = dfExport.dropna(subset=['summary']) 

dfExport.reset_index(drop=True, inplace=True)

dfExport.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506438 entries, 0 to 506437
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   uuid         506438 non-null  object        
 1   overall      506438 non-null  float64       
 2   verified     506438 non-null  bool          
 3   reviewTime   506438 non-null  datetime64[ns]
 4   reviewYear   506438 non-null  int64         
 5   reviewerID   506438 non-null  object        
 6   asin         506438 non-null  object        
 7   style        215400 non-null  object        
 8   reviewText   506438 non-null  object        
 9   summary      506438 non-null  object        
 10  vote         103930 non-null  object        
 11  category     506438 non-null  string        
 12  description  506438 non-null  object        
 13  title        506438 non-null  object        
 14  brand        506438 non-null  object        
 15  feature      506438 non-null  obje

In [28]:
dfExport.to_pickle(jarvis.DATA_DIR + '/02_working/01_Cellphone_full.pkl', protocol=4)

In [30]:
dfExport['overall_posneg'] = dfExport.overall.apply(lambda x: 1 if x > 3 else 0)

In [32]:
cwutils.expandColumn(dfExport, 'reviewText')

Expanding column: reviewText: 506438it [54:48, 154.02it/s]


In [33]:
#Bert Encode Column
cwutils.columnEncode(data=dfExport,
                   columnName='reviewText_lemma',
                   #transformerType='sentence-transformers/distilbert-base-nli-mean-token',
                    transformerType = 'sentence-transformers/multi-qa-distilbert-cos-v1',
                   colSuffix='_bert')


Loading sentence transformer: sentence-transformers/multi-qa-distilbert-cos-v1
sentence-transformers/multi-qa-distilbert-cos-v1 sentence transformer has been loaded
Generating encodings


100%|█████████████████████████████████| 506438/506438 [2:38:46<00:00, 53.16it/s]

sentence-transformers/multi-qa-distilbert-cos-v1 encodings completed


In [34]:
dfExport.to_pickle(jarvis.DATA_DIR + '/02_working/01_Cellphone_full.pkl.gz', protocol=4)

In [38]:
#jarvis.showProjectDataFiles()
jarvis.compressProjectDataFiles(removeOriginal=True)

Scanning [/home/magni//ML_Root/project_root/data/ML1010-Group-Project] for data files to compress
======> No files found in directory.

Scanning [/home/magni//ML_Root/project_root/data/ML1010-Group-Project/01_original] for data files to compress
======> 3 files found in directory.
======> 0 file(s) have been compressed.

Scanning [/home/magni//ML_Root/project_root/data/ML1010-Group-Project/02_working] for data files to compress
======> 40 files found in directory.
======> compressing file: 01_Cellphone_full.pkl
======> compressing file: 01_Cellphone_small.pkl
======> compressing file: 01_NLP_ReviewText_small.pkl
======> compressing file: 01_NLP_Summary_small.pkl
======> compressing file: 01_NLP_Title_small.pkl
======> compressing file: 02_Cellphone.pkl
======> compressing file: 02_NLP_ReviewTextData.pkl
======> compressing file: 02_NLP_SummaryData.pkl
======> compressing file: 02_NLP_TitleData.pkl
======> compressing file: 03_Cellphone.pkl
======> compressing file: 03_NLP_ReviewTextDat